# Topic Modeling and Gibbs Sampling

Задача: описать текст через распределение весов по некоторому фиксированному набору топиков (тегов). Например, для набора тегов Политика, Военные сражения, Спорт, Интернет, Драма представить роман "Война и мир" как вектор (0.3, 0.2, 0, 0, 0.5), а статью в газете про допинг в велоспорте как вектор (0.1, 0, 0.7, 0, 0.2).

Для чего, например, это нужно: имея векторное представление для текстов, тексты можно сравнивать, рекомендовать похожие.

Условие: даны только набор текстов и количество тем.

## Немного теории

Будем представлять текст как неупорядоченный набор слов (Bag-of-words model). Предположим, что имеется K тегов и для каждого тега выбрано распределение $\phi_k$ над списком всевозможных слов (словарем из N слов). По сути, каждое $\phi_k$ - это вектор длины N из неотрицательных величин, в сумме дающих 1. Вектора $\phi_k$ независимы и моделируются распредеделением Дирихле $Dir(\beta)$. Теперь, чтобы собрать текст d из $n$ слов, будем действовать по следующей схеме:


* выберем распредление для тегов $\theta_d$. Вновь, $\theta_d$ - это вектор длины K из неотрицательных величин, в сумме дающих 1. Поэтому естественно брать $\theta_d \sim Dir(\theta | \alpha)$

* Для i от 1 до n:
  * выберем тег $z_i$ согласно распределению $\theta_d$
  * выберем слово $w_i$ из распределения для данного тега, т.е. $w_i \sim \phi_{z_i}$
  * добавляем слово $w_i$ в текст.

Полученная модель называется моделью LDA (Latent Dirichlet Allocation). Описанная  схема задает совместное распределение скрытых и наблюдаемых параметров по всем текстам корпуса размера M в виде:


$p(\textbf{w}, \textbf{z}, \theta, \phi | \alpha, \beta) = Dir(\theta | \alpha) Dir(\phi|\beta)Cat(\textbf{z}|\theta)Cat(\textbf{w}|\phi_z)$.

Здесь $\textbf{w}$ и $\textbf{z}$ обозначают вектора слов и тегов по всем текстам, $\theta$ - набор из $\theta_d$ для каждого документа (матрица $M\times K$), $\phi$ - набор из $\phi_k$ для каждого тега (матрица $K\times N$).


![img](https://www.researchgate.net/publication/336065245/figure/fig1/AS:807371718815752@1569503826964/Latent-Dirichlet-allocation-LDA-process-and-its-two-outputs-a-LDA-document.ppm)

Наша задача - восстановить распределение $p(\textbf{z}, \theta, \phi | \textbf{w}, \alpha, \beta)$.


Немного упростим жизнь, и поставим себе задачей восстановить распределение $p(\textbf{z} | \textbf{w}, \alpha, \beta) = \int\int p(\textbf{z}, \theta, \phi | \textbf{w}, \alpha, \beta)\textrm{d}\theta\textrm{d}\phi$.

В этот момент на помощь приходить алгоритм Gibbs Sampling. Напомним, для оценки набора парамеров $\textbf{z} = (z_1, z_2, ..., z_m)$ используется схема:

$z_i^{(t)} \sim p(z_i^{(t)}\ | \ z_1=z_1^{t}, ..., z_{i-1}=z_{i-1}^{t},
z_{i+1}=z_{i+1}^{t-1}, z_{m}=z_{m}^{t-1})$.

Условные распределения выводятся так. Сначала замечаем, что

$p(z_i|\textbf{z}_{\hat{i}}, \textbf{w}, \alpha, \beta) = \frac{p(z_i,\textbf{z}_{\hat{i}}, \textbf{w}| \alpha, \beta)}{p(\textbf{z}_{\hat{i}}, \textbf{w}| \alpha, \beta)}  = \frac{p(\textbf{z}, \textbf{w}| \alpha, \beta)}{p(\textbf{z}_{\hat{i}}, \textbf{w}_{\hat{i}}| \alpha, \beta)  p(w_i|\alpha, \beta)}$.

Здесь $\textbf{z}_{\hat{i}}$, $\textbf{w}_{\hat{i}}$ - вектора без $i$-oй копмоненты. 

Далее расписываем:

$p(\textbf{z}, \textbf{w}| \alpha, \beta) = \int\int p(\textbf{z}, \textbf{w}, \theta, \phi| \alpha, \beta)\textrm{d}\theta\textrm{d}\phi = \int\int Dir(\theta | \alpha) Dir(\phi|\beta)Cat(\textbf{z}|\theta)Cat(\textbf{w}|\phi_z)\textrm{d}\theta\textrm{d}\phi = 
\int Dir(\theta | \alpha) Cat(\textbf{z}|\theta)\textrm{d}\theta \int Dir(\phi|\beta)Cat(\textbf{w}|\phi_z)\textrm{d}\phi$

и обнаруживаем, что оба интеграла в последнем выражении вычисляются аналитически. Для примера первый:

$\int Dir(\theta | \alpha) Cat(\textbf{z}|\theta)\textrm{d}\theta = \prod\limits_d \int Dir(\theta_d | \alpha) Cat(\textbf{z}_d|\theta_d)\textrm{d}\theta_d = \prod\limits_d \int \frac{1}{B(\alpha)}\prod\limits_k \theta_{d, k}^{\alpha-1}\prod\limits_i \theta_{d, z_i}\textrm{d}\theta_d = \prod\limits_d\frac{1}{B(\alpha)}\int\prod\limits_k \theta_{d, k}^{n_{d, k} + \alpha - 1}\textrm{d}\theta_d = \prod\limits_d \frac{B(n_{d,\cdot} + \alpha)}{B(\alpha)}$.

Здесь  $n_{d,k}$ - количество тэгов $k$ в тексте $d$, $n_{d,\cdot}$ - вектор длины $K$ из этих величин.

Аналогично, второй интеграл 
$\int Dir(\phi|\beta)Cat(\textbf{w}|\phi_z)\textrm{d}\phi = \prod\limits_k \frac{B(n_{k,\cdot} + \beta)}{B(\beta)}$,

где $n_{k,\cdot}$ - вектор длины $N$ встречаемости слов внутри тэга $k$.

Получаем: 

$p(\textbf{z}, \textbf{w}| \alpha, \beta) = \prod\limits_d \frac{B(n_{d,\cdot} + \alpha)}{B(\alpha)} \prod\limits_k \frac{B(n_{k,\cdot} + \beta)}{B(\beta)}$.

Теперь
$p(z_i|\textbf{z}_{\hat{i}}, \textbf{w}, \alpha, \beta) \propto \prod\limits_d \frac{B(n_{d,\cdot} + \alpha)}{B(n_{d,\cdot}^{\hat{i}} + \alpha)} \prod\limits_k \frac{B(n_{k,\cdot} + \beta)}{B(n_{k,\cdot}^{\hat{i}} + \beta)}$.

Знак $\propto$ означает пропорциональность с точностью до общего множителя $p(w_i|\alpha, \beta)$. Векторы $n_{d,\cdot}^{\hat{i}}$ и $n_{k,\cdot}^{\hat{i}}$ получены из векторов $n_{d,\cdot}$ и $n_{k,\cdot}$ после выбрасывания $z_i$. 

Выражение упрощается дальше, расписывая бета-функцию через гамма-функции. Напомним,
$B(x_1, ..., x_m) = \frac{\Gamma(x_1)\cdot...\cdot\Gamma(x_m)}{\Gamma(x_1 + ... + x_m)}$, а также $\Gamma(n) = (n-1)\Gamma(n-1)$. Получим:

$p(z_i=k |\textbf{z}_{\hat{i}}, \textbf{w}, \alpha, \beta) \propto (n_{d_i, k}^{\hat{i}} + \alpha_k) \frac{n_{k, w_i}^{\hat{i}} + \beta_{w_i}}{\sum\limits_{w}(n_{k, w}^{\hat{i}} + \beta_{w})}$.


С этого места можно полностью собрать алгоритм моделирования плотности $p(\textbf{z}| \textbf{w}, \alpha, \beta)$. Введем  обозначение $n_k$ - количество слов, отнесенных к тегу $k$, $W$ - общее количество слов в корпусе, $\beta_{sum} = \sum\limits_w\beta_w$

Алгоритм:

* заведем счетчики $n_{k, w}$, $n_{d, k}$, $n_k$
* случайным образом расставим теги словам, обновим счетчики $n_{k, w}$, $n_{d, k}$, $n_k$
* пока не сойдемся к стационарному режиму:
  * для каждого $i$ от 1 до $W$:
      * для каждого $k$ от 1 до $K$:
        * $I = I\{z_i = k\}$ (индикатор)
        * вычисляем $p_k = (n_{d_i, k} + \alpha_k - I) \frac{n_{k, w_i} + \beta_{w_i} - I}{n_k + \beta_{sum} - I}$
      * сэмплим новый $z_i$ из полученного распределения $(p_1, ..., p_K)$
      * обновляем счетчики для учета обновленого значения $z_i$

На практике удобно реализовавать так:

* заведем счетчики $n_{k, w}$, $n_{d, k}$, $n_k$, заполненные нулями
* случайным образом расставим теги словам, обновим счетчики $n_{k, w}$, $n_{d, k}$, $n_k$
* пока не сойдемся к стационарному режиму:
  * для каждого $i$ от 1 до $W$:
      * $n_{d_i, z_i} \mathrel{-}= 1$, $n_{z_i, w_i} \mathrel{-}= 1$, $n_{z_i} \mathrel{-}= 1$
      * для каждого $k$ от 1 до $K$:
        * вычисляем $p_k = (n_{d, k} + \alpha_k) \frac{n_{k, w_i} + \beta_{w_i}}{n_k + \beta_{sum}}$
      * сэмплим новый $z_i$ из полученного распределения $(p_1, ..., p_K)$
      * $n_{d_i, z_i} \mathrel{+}= 1$, $n_{z_i, w_i} \mathrel{+}= 1$, $n_{z_i} \mathrel{+}= 1$





Восстановив распредление для $\textbf{z}$, можем оценить $\theta$ и $\phi$, о которых мы ненадолго забыли. Оценить можно, например, через матожидание по апостериорным распределениям. Получите формулы самостоятельно!

Литература:

http://u.cs.biu.ac.il/~89-680/darling-lda.pdf

https://www.cs.cmu.edu/~mgormley/courses/10701-f16/slides/lecture20-topic-models.pdf

Перейдем к практике.

## Датасет

Возьмем популярный датасет [20 Newsgroups](http://qwone.com/~jason/20Newsgroups/), встроенный в пакет ```sklearn```. Датасет состоит из ~20К текстов, классифицированных на 20 категорий. Датасет разбит на ```train``` и ```test```. Для загрузки используем  модуль ```fetch_20newsgroups```, в параметрах указать, что мета информацию о тексте загружать не нужно:

In [3]:
import numpy as np
from sklearn.datasets import fetch_20newsgroups

newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))

Выведем список категорий текстов:

In [4]:
newsgroups_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

Атрибут ```traget``` хранит номера категорий для текстов из обучающей выборки:

In [5]:
newsgroups_train.target[:10]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4])

Доступ к самим текстам через атрибут ```data```. Выведем текст и категорию случайного примера из обучающего датасета:

In [6]:
n = 854
print('Topic = {0}\n'.format(newsgroups_train.target_names[newsgroups_train.target[n]]))
print(newsgroups_train.data[n])

Topic = rec.motorcycles

hey... I'm pretty new to the wonderful world of motorcycles... I just
bought
a used 81 Kaw KZ650 CSR from a friend.... I was just wondering what kind of

saddle bags I could get for it (since I know nothing about them)  are there
bags for the gas tank?  how much would some cost, and how much do they
hold?
thanks for your advice!!!  I may be new to riding, but I love it
already!!!!
:)




## Векторное представление текста

Представим текст как вектор индикаторов вхождений слов из некоторого словаря в текст. Это простейшая модель BOF. 

Сформируем словарь на основе нашего набора текстов. Для этого используем модуль ```CountVectorizer```:

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS

vectorizer = CountVectorizer(lowercase=True, stop_words=ENGLISH_STOP_WORDS,
                             analyzer='word', binary=True, min_df=12, max_df=0.05)
vectorizer.fit(newsgroups_train.data)

CountVectorizer(analyzer='word', binary=True, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.05, max_features=None, min_df=12,
        ngram_range=(1, 1), preprocessor=None,
        stop_words=frozenset({'must', 'six', 'those', 'before', 'whereby', 'while', 'down', 'nine', 'always', 'i', 'cannot', 'few', 'ever', 'hence', 'but', 'everywhere', 'me', 'anyhow', 'as', 'latterly', 'towards', 'whatever', 'fifteen', 'hundred', 'my', 'nobody', 'whenever', 'fifty', 'eleven', 'fire', 'how...e', 'per', 'well', 'side', 'your', 'becoming', 'hasnt', 'no', 'around', 'serious', 'empty', 'bill'}),
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

Количество проиндексированных слов:

In [8]:
len(vectorizer.vocabulary_)

8963

Проиндексированные слова и их индексы:

In [9]:
vectorizer.vocabulary_

{'wondering': 8843,
 'enlighten': 3078,
 'car': 1612,
 'saw': 7149,
 'door': 2829,
 'sports': 7651,
 'looked': 4949,
 'late': 4746,
 '60s': 446,
 'early': 2925,
 '70s': 482,
 'doors': 2830,
 'small': 7513,
 'addition': 662,
 'bumper': 1513,
 'separate': 7294,
 'rest': 6893,
 'body': 1370,
 'model': 5327,
 'engine': 3065,
 'specs': 7619,
 'production': 6369,
 'history': 4030,
 'info': 4314,
 'fair': 3323,
 'brave': 1441,
 'souls': 7576,
 'upgraded': 8477,
 'si': 7411,
 'clock': 1859,
 'oscillator': 5797,
 'shared': 7352,
 'experiences': 3244,
 'poll': 6173,
 'send': 7277,
 'brief': 1456,
 'message': 5208,
 'procedure': 6352,
 'speed': 7625,
 'cpu': 2308,
 'rated': 6608,
 'add': 659,
 'cards': 1618,
 'adapters': 656,
 'heat': 3973,
 'hour': 4093,
 'usage': 8493,
 'floppy': 3494,
 'disk': 2753,
 'functionality': 3629,
 '800': 500,
 'floppies': 3493,
 'especially': 3128,
 'requested': 6848,
 'days': 2454,
 'network': 5538,
 'knowledge': 4693,
 'base': 1200,
 'upgrade': 8476,
 'haven': 3945

Индекс, например, для слова anyone:

In [10]:
vectorizer.vocabulary_.get('car')

1612

А теперь преобразуем строку в вектор:

In [11]:
text = 'I was wondering if anyone out there could enlighten me on this car I saw'
x = vectorizer.transform([text])

Какой тип имеет объект, на который указывает ```x```?

In [12]:
type(x)

scipy.sparse.csr.csr_matrix

Разреженная матрица!

### Отступление про разреженные матрицы

Список ненулевых элементов матрицы:

In [13]:
x.data

array([1, 1, 1, 1])

Индексы строк и столбцов для ненулевых элементов:

In [14]:
x.nonzero()

(array([0, 0, 0, 0], dtype=int32),
 array([1612, 3078, 7149, 8843], dtype=int32))

Преобразование к объекту ndarray (именно после приведения к такому виду разреженные матрицы можно подставлять в функции, например, библиотеки Numpy):

In [15]:
x.toarray()

array([[0, 0, 0, ..., 0, 0, 0]])

Вернемся к словарю. Раскодируем вектор ```x``` в список слов:

In [16]:
vectorizer.inverse_transform(x)

[array(['car', 'enlighten', 'saw', 'wondering'], dtype='<U79')]

Пропало слово ```I```. Но дело в том, что по умолчанию ```CountVectorizer``` отбрасывает последовательности, короче 2 символов. На это указывает параметр ```token_pattern='(?u)\\b\\w\\w+\\b'```.

Переведем весь набор текстов обучающего датасета в набор векторов, получим матрицу ```X_train```:

In [17]:
X_train = vectorizer.fit_transform(newsgroups_train.data)
X_train.shape

(11314, 8963)

О пользе разреженных матриц. Отношение числа ненулевых элементов ко всем элементам матрицы ```X_train```:

In [18]:
X_train.nnz / np.prod(X_train.shape)

0.004860444972339391


Задача: запустить модель LDA и Gibbs Sampling с числов тегов 20. Вывести топ-10 слов по каждому тегу. Соотнести полученные теги с тегами из датасета, сделать выводы. 

In [19]:
from tqdm import tqdm

In [20]:
topics = 20
n_k=np.zeros(topics)
n_dk=np.zeros(topics*X_train.shape[0]).reshape(X_train.shape[0],topics)
n_kw=np.zeros(topics*X_train.shape[1]).reshape(topics,X_train.shape[1])
doc, word = X_train.nonzero()
z = np.random.choice(topics, len(doc))
    
for i, j, k in zip(doc, word, z):
    n_dk[i, k] += 1
    n_kw[k, j] += 1
    n_k[k] += 1

In [21]:
def lda(n_dk, n_kw, n_k, z, doc, word, topics, alpha, beta, it):    
    for i in tqdm(range(it)):
        for j in range(len(doc)):
            cur_word = word[j]
            cur_doc = doc[j]
            cur_topic = z[j]
            
            n_dk[cur_doc, cur_topic] -= 1
            n_kw[cur_topic, cur_word] -= 1
            n_k[cur_topic] -= 1
            
            p = (n_dk[cur_doc, :] + alpha) * (n_kw[:, cur_word] + beta[cur_word]) / (n_k + beta.sum())
            z[j] = np.random.choice(np.arange(topics), p=p / p.sum())
            
            n_dk[cur_doc, z[j]] += 1
            n_kw[z[j], cur_word] += 1
            n_k[z[j]] += 1
    return z, n_kw, n_dk, n_k

In [22]:
z,n_kw, n_dk, n_k=lda(n_dk, n_kw, n_k, z, doc, word, 20, 2*np.ones(20), 2*np.ones(X_train.shape[1]), 50)

100%|██████████| 50/50 [14:55<00:00, 17.91s/it]


In [23]:
answer = np.argsort(n_kw, axis=1)[:, -10:]
for i in range(20):
    matrix = np.zeros((1, X_train.shape[1]))
    for j in answer[i]:
        matrix[0, j] = 1
    print('\t'.join(vectorizer.inverse_transform(matrix)[0]))

11	12	14	24	25	50	ah	ca	hi	ii
ago	big	book	car	hand	makes	oh	small	stuff	trying
banks	geb	gordon	intellect	n3jxp	pitt	shameful	skepticism	soon	surrender
article	couple	folks	pretty	remember	seen	similar	stuff	support	worth
goes	guess	knows	non	pretty	problems	reason	remember	seen	yes
ask	car	cause	david	isn	left	nice	ok	pretty	price
earth	launch	low	nasa	sale	science	sell	small	space	university
computer	early	general	interested	issue	order	place	small	state	test
ago	doing	email	fine	goes	instead	isn	place	seen	won
exactly	getting	haven	heard	local	money	similar	start	thought	wait
bible	christian	claim	jesus	life	man	mean	person	reason	says
ago	buy	difference	doing	guess	hi	seen	thank	usually	won
bad	car	email	friend	getting	info	problems	seen	start	unless
algorithm	chip	clipper	data	encryption	government	key	keys	phone	public
anybody	big	drive	experience	gets	getting	heard	interested	looks	wouldn
11	12	13	15	20	30	game	games	play	team
away	children	dead	government	israel	jews	killed	le